In [1]:
from __future__ import division
import os
import sys
import glob

import numpy as np
from IPython.core.display import Image
import uuid 

import gc

In [2]:
#R call

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
fastclime = importr('fastclime')
grdevices = importr('grDevices')
base = importr('base')
stats = importr('stats')

In [3]:
#Call custom Python module
import parametric as param

In [4]:
def is_Hermitian(m):
    #Set missing to zero prior to checking symmetry
    m[np.isnan(m)] = 0.

    try:
        return np.allclose(np.transpose(m,(1,0)), m)
    except:
        return False

In [5]:
def fastclime_main(x,lambda_min=0.1,nlambda=50):

    cov_input = 1
    SigmaInput = x.copy()
    
    #Check if matrix is symmetric
    if not is_Hermitian(SigmaInput):
        SigmaInput = np.corrcoef(SigmaInput.T)
        cov_input = 0
    
    #Run parametric simplex linear solver
    Sigmahat, mu, maxnlambda, iicov = param.mainfunc(SigmaInput,lambda_min,nlambda)
      
    #Process output
    maxnlambda+=1
    
    #Reshape the array in Fortran order
    #and then slice the array to extract only the top maxnlambda rows
    lambdamtx = mu.T.reshape(nlambda, -1, order='F')[:maxnlambda,:]
    mu = None
    
    #Take each row of iicov and convert it to a d x d matrix
    d = Sigmahat.shape[1]
    icovlist = np.empty((d, d, maxnlambda)) 
    
    for i in range(maxnlambda):
        icovlist[:,:,i] = iicov[:,i].reshape((d,d)).T
    
    gc.collect()
    
    return x, cov_input, Sigmahat, maxnlambda, lambdamtx, icovlist
    

In [6]:
# #Generate data

L = fastclime.fastclime_generator(n = 100, d = 20)
pydat = np.array(L.rx2('data'))

Generating data from the multivariate normal distribution with the random graph structure....done.


In [7]:
Pyout = fastclime_main(pydat,0.1)

In [8]:
#Compare results to R

In [9]:
#%timeit -n1 -r1 fastclime.fastclime(L.rx2('data'),0.1)
Rout = fastclime.fastclime(L.rx2('data'),0.1)

Allocating memory 
start recovering 
preparing precision and path matrix list 
Done! 


In [10]:
from random import randint
ranint = randint(0,Pyout[3]-1)
print np.isclose(Pyout[3],np.array(Rout.rx2('maxnlambda')))
print np.allclose(Pyout[4],np.array(Rout.rx2('lambdamtx')))
print np.allclose(Pyout[5][:,:,ranint],np.array(Rout.rx2('icovlist')[ranint]))

[ True]
True
True
